In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, FloatType, BooleanType, StringType
from pyspark.sql import functions as f
from pyspark.sql.functions import when

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/03 14:14:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
d = '/Users/akot/Study/iw/carto/input_data'

In [37]:
import os

heads = [os.path.join(d, x) for x in os.listdir(d) if x.endswith('_00.csv')]
not_head = [os.path.join(d, x) for x in os.listdir(d) if not x.endswith('_00.csv')]

In [38]:
DEFAULT_SCHEMA = StructType([
    StructField("VendorID", IntegerType(), nullable=False),
    StructField("tpep_pickup_datetime", TimestampType(), nullable=False),
    StructField("tpep_dropoff_datetime", TimestampType(), nullable=False),
    StructField("passenger_count", IntegerType(), nullable=False),
    StructField("trip_distance", FloatType(), nullable=False),
    StructField("pickup_longitude", FloatType(), nullable=False),
    StructField("pickup_latitude", FloatType(), nullable=False),
    StructField("RateCodeID", IntegerType(), nullable=False),
    StructField("store_and_fwd_flag", StringType(), nullable=False),
    StructField("dropoff_longitude", FloatType(), nullable=False),
    StructField("dropoff_latitude", FloatType(), nullable=False),
    StructField("payment_type", IntegerType(), nullable=False),
    StructField("fare_amount", FloatType(), nullable=False),
    StructField("extra", FloatType(), nullable=False),
    StructField("mta_tax", FloatType(), nullable=False),
    StructField("tip_amount", FloatType(), nullable=False),
    StructField("tolls_amount", FloatType(), nullable=False),
    StructField("improvement_surcharge", FloatType(), nullable=False),
    StructField("total_amount", FloatType(), nullable=False),
])

In [39]:
df_heads = (spark
          .read
          .options(header=True, delimiter='\t', mode="DROPMALFORMED")
          .schema(schema=DEFAULT_SCHEMA)
          .csv(heads))

df = (spark
          .read
          .options(header=False, delimiter='\t', mode="DROPMALFORMED")
          .schema(schema=DEFAULT_SCHEMA)
          .csv(not_head))

In [40]:
df = df.union(df_heads)

In [41]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'RateCodeID',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount']

In [42]:
df.select("RateCodeID").distinct().show()

+----------+
|RateCodeID|
+----------+
|         1|
|         6|
|         3|
|         5|
|         4|
|         2|
|        99|
|      null|
+----------+



In [31]:
df.select("payment_type").distinct().show()

+------------+
|payment_type|
+------------+
|           1|
|           3|
|           4|
|           2|
|           5|
|        null|
+------------+



In [65]:
cols = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

res = [
    (col, df.filter( (f.col(col) < -90) | (f.col(col) > 90)).count(), df.filter( (f.col(col) == 0) ).count())
    for col in cols
]

res  # col_name, number of x < -90 | x > 90, x == 0

[('pickup_longitude', 55, 613320),
 ('pickup_latitude', 1, 613316),
 ('dropoff_longitude', 62, 593219),
 ('dropoff_latitude', 4, 593219)]